In [ ]:
# imports
import numpy as np
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
from sklearn.feature_selection import SelectKBest, f_classif
df = pd.read_csv('mammographic_masses_data.csv')


In [ ]:
# 1.1 Showing the first 5 rows of the dataset
df.head()
df.describe()

In [ ]:
# 2.2 Show the points in the dataset where the Severity is 1
loc_df = df.loc[df['Severity'] == 1 , 'BA'] #can choose to only show specific characteristics with severity == 1
print(loc_df)

In [ ]:
ageHist = px.histogram(df[['Age']])
baScatter = px.scatter(df, x='BA', y='Age', color='BA', color_continuous_scale=px.colors.sequential.Bluered_r)
baScatter.show()
ageHist.show()

In [ ]:
#3.1


df_cp = df.copy()
# Removes empty cells. inplace=True makes sure to create a new dataset with the changes saved
df_cp.dropna(inplace=True)


ageHist_cp = px.histogram(df_cp[['Age']])
baScatter_cp = px.scatter(df_cp, x='BA', y='Age', color='BA', color_continuous_scale=px.colors.sequential.Bluered_r)

# Making side by side plots 
sbs = make_subplots(rows=1, cols=2, subplot_titles=
        ("Before dropna", "After dropna"))
# Creating
sbs.update_layout(height=600, width=1800, title_text="Side by Side Visualizations")

# OG version before dropna function
hist = ageHist.data[0]
# after dropna function
hist_cp = ageHist_cp.data[0]

scatter = baScatter_cp.data[0]

# Adding graphs to side side by side graph 
# sbs.add_trace(hist, row=1, col=1)
# sbs.add_trace(hist_cp, row=1, col=2)
# sbs.show()


# ageHist.show()
# baScatter.show()

# 3.2

# Sinlge column normalization
def lin_norm(val, col):
    val = val[[col]]
    max_val = val.max()
    min_val = val.min()
    return (val - min_val) / (max_val - min_val)

# Square Root Normalization
def sqrt_norm(val, col):
    val = val[[col]]
    max_val = val.max()
    min_val = val.min()
    return np.sqrt((val - min_val) / (max_val - min_val))

# Logarithmic Normalizatio
def log_norm(val, col):
    val = val[[col]]
    min_val = val.min()
    max_val = val.max()
    return (np.log(val) - np.log(min_val)) /(np.log(max_val  - np.log(min_val)))

# df_lin_norm = lin_norm(df_cp, 'Age')
# df_lin_norm[['Age']].head()

df_lin_norm_test = lin_norm(df_cp, 'Age')
df_lin_norm_test.head()

# df_sqrt_norm = sqrt_norm(df_cp)
# df_sqrt_norm.head()

# df_log_norm = log_norm(df_cp)
# df_log_norm.head()









In [ ]:
#4
